In [29]:
import math
import os
import subprocess
import uuid

import cv2
import cv2 as cv
import matplotlib.cm
import numpy as np
import PIL
import scipy
import skimage
import skimage.segmentation
import sklearn
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage import feature, measure
from skimage.feature import blob_dog, blob_doh, blob_log
from skimage.filters import threshold_local
from sklearn import mixture
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

## Set up dataset

In [2]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import livecellx
from livecellx.segment import datasets
import livecellx.segment
from livecellx import segment

seg_path = r"""D:/xing-vimentin-dic-pipeline/src/livecell_dev/notebooks/segmentation_results/detectron_model3/restart_day0_Group 1_wellA1_RI_MIP_stitched"""
label_seg_imgs = core.datasets.LiveCellImageDataset(seg_path, ext="png")
dir_path = Path(
    r"D:/xing-vimentin-dic-pipeline/src/livecell_dev/cxa-data/june_2022_data_8bit_png/restart_day2_Group 1_wellA1_RI_MIP_stitched"
)
raw_imgs = core.datasets.LiveCellImageDataset(dir_path, ext="png")

576 png img file paths loaded: 
576 png img file paths loaded: 


In [3]:
np.where(label_seg_imgs[0].flatten() > 1) # check if it is a binary mask?

(array([   3847,    3868,    3869, ..., 4897017, 4897018, 4897019],
       dtype=int64),)

In [4]:
len(set(label_seg_imgs[0].flatten()))

28

In [5]:
regions = measure.regionprops(label_seg_imgs[0])

In [6]:
len(regions)

27

In [7]:
regions[0].bbox

(152, 190, 359, 445)

In [8]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
def createTrackerByName(trackerType):
  """adapted from https://learnopencv.com/multitracker-multiple-object-tracking-using-opencv-c-python/"""
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.legacy.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]:
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)

  return tracker
  
def get_bbox_from_regionprops_skimge(regions):
    """Get bounding box from regionprops.
    
    Parameters
    ----------
    regions : list of skimage.measure._regionprops._RegionProperties
        List of region properties.
    
    Returns
    -------
    list of tuple
        List of bounding boxes.
    """
    return [region.bbox for region in regions]

def bbox_skimage_to_cv2_order(bboxes):
    """Convert bounding box from skimage order to cv2 order.
    
    Parameters
    ----------
    bboxes : list of tuple
        List of bounding boxes.
    
    Returns
    -------
    list of tuple
        List of bounding boxes.
    """
    return [(bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]) for bbox in bboxes]

def gray_img_to_rgb(img):
    """Convert gray image to rgb image.
    
    Parameters
    ----------
    img : np.ndarray
        Gray image.
    
    Returns
    -------
    np.ndarray
        RGB image.
    """
    return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

def get_cv2_bbox(label_mask: np.array):
  """generate cv2 style bounding box from label mask

  Parameters
  ----------
  label_mask :
    label mask: W x H np array with each pixel value indicating the label of objects (index 1, 2, 3, ...). Note that labels are not required to be consecutive.. 

  Returns
  -------
  _type_
      _description_
  """
  regions = measure.regionprops(label_mask)
  bboxes = get_bbox_from_regionprops_skimge(regions)
  bboxes_cv2 = bbox_skimage_to_cv2_order(bboxes)
  return bboxes_cv2

In [9]:
cv2.__version__

'4.6.0'

In [10]:
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

# Specify the tracker type
# trackerType = "CSRT"    
trackerType = "BOOSTING"

# Create MultiTracker object
multiTracker = cv2.legacy.MultiTracker()


bboxes_cv2 = get_cv2_bbox(label_seg_imgs[0])

# Initialize MultiTracker
# if we do not use 3 channel image, c++ opencv will compain about it.
# https://github.com/GuiltyNeuron/ANPR/issues/3
rgb_init_img = gray_img_to_rgb(raw_imgs[0]) # np.repeat(raw_imgs[0][:, :, np.newaxis], 3, axis=2)

# initialize the tracker with first frame's bounding boxes
for bbox in bboxes_cv2:
  multiTracker.add(createTrackerByName(trackerType), rgb_init_img, bbox)


In [11]:
multiTracker.getObjects().shape

(27, 4)

In [12]:
multiTracker.getObjects()[:2]

array([[ 152.,  190.,  207.,  255.],
       [1297., 1159.,  274.,  322.]])

In [13]:
multiTracker.update(gray_img_to_rgb(raw_imgs[1]));

In [14]:
multiTracker.getObjects()[:2]

array([[ 151.,  178.,  207.,  255.],
       [1301., 1159.,  274.,  322.]])

In [16]:
multiTracker.getObjects()

array([[1.510e+02, 1.780e+02, 2.070e+02, 2.550e+02],
       [1.301e+03, 1.159e+03, 2.740e+02, 3.220e+02],
       [0.000e+00, 1.386e+03, 2.320e+02, 1.520e+02],
       [1.792e+03, 1.254e+03, 2.260e+02, 2.240e+02],
       [3.580e+02, 1.800e+02, 2.340e+02, 2.330e+02],
       [6.500e+01, 4.270e+02, 5.380e+02, 2.700e+02],
       [1.802e+03, 1.491e+03, 2.510e+02, 8.000e+01],
       [7.810e+02, 2.232e+03, 4.100e+01, 1.500e+01],
       [9.870e+02, 2.289e+03, 1.520e+02, 1.200e+02],
       [1.735e+03, 1.573e+03, 2.560e+02, 1.190e+02],
       [1.070e+02, 1.305e+03, 2.190e+02, 1.020e+02],
       [1.006e+03, 2.324e+03, 9.000e+01, 6.400e+01],
       [7.610e+02, 2.237e+03, 2.270e+02, 1.410e+02],
       [7.000e+00, 1.235e+03, 1.080e+02, 7.000e+01],
       [2.550e+02, 4.630e+02, 8.000e+01, 6.400e+01],
       [1.500e+01, 1.233e+03, 2.890e+02, 1.660e+02],
       [1.858e+03, 1.349e+03, 5.400e+01, 5.000e+01],
       [5.900e+01, 1.390e+03, 7.600e+01, 5.900e+01],
       [5.900e+01, 1.262e+03, 8.800e+01, 6.300

In [33]:
multiTracker.getObjects()[:2]

array([[ 222.,  154.,  207.,  255.],
       [1303., 1162.,  274.,  322.]])

In [34]:
multiTracker.update(gray_img_to_rgb(np.zeros(raw_imgs[0].shape, dtype=np.uint8)));

In [36]:
multiTracker.getObjects()[:2]

array([[ 139.,   52.,  207.,  255.],
       [1193., 1033.,  274.,  322.]])

In [38]:
from livecellx.core import SingleCellStatic, SingleCellTrajectory
rgb_init_img = gray_img_to_rgb(raw_imgs[0]) # np.repeat(raw_imgs[0][:, :, np.newaxis], 3, axis=2)
# Create MultiTracker object
multiTracker = cv2.legacy.MultiTracker()

# initialize the tracker with first frame's bounding boxes
for bbox in bboxes_cv2:
  multiTracker.add(createTrackerByName(trackerType), rgb_init_img, bbox)

trajectories = []
_bboxes_cv2_init = multiTracker.getObjects()
for i, obj in enumerate(_bboxes_cv2_init):
    single_cell_trajectory = SingleCellTrajectory(raw_imgs)
    trajectories.append(single_cell_trajectory)


for idx in tqdm(range(len(raw_imgs))):
    img = raw_imgs[idx]
    label_mask = label_seg_imgs[idx]
    bboxes_cv2_current = get_cv2_bbox(label_mask)
    multiTracker.update(gray_img_to_rgb(img))
    
    tracked_bboxes_cv2 = multiTracker.getObjects()
    for i, obj in enumerate(tracked_bboxes_cv2):
        single_cell_trajectory = SingleCellTrajectory(raw_imgs)
        trajectories.append(single_cell_trajectory)
    SingleCellStatic(obj, timeframe=idx, regionprops=regions[i], img_dataset=raw_imgs)
    tracked_bboxes = multiTracker.getObjects()
    break # for test

  0%|          | 0/576 [00:01<?, ?it/s]
